In [1]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [2]:
background = None
average_weight = 0.5
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [3]:
def calc_avg(frame,average_weight):
    global background
    if background is None:
        background = frame.copy().astype('float')
        return None
    cv2.accumulateWeighted(frame,background,average_weight)

In [4]:
def segment(frame,th=25):
    global background
    diff = cv2.absdiff(background.astype('uint8'),frame)
    ret, th_img = cv2.threshold(diff,th,255,cv2.THRESH_BINARY)
    image,contours,hierarchy = cv2.findContours(th_img.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours)==0:
        return None
    else:
        #ASSUMING LARGEST EXTERNAL CONT IN ROI IS HAND
        hand_segment = max(contours,key=cv2.contourArea)
        return (th_img,hand_segment)

In [5]:
def count_fingers(th_img,hand_segment):
    conv_hull = cv2.convexHull(hand_segment)
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
    
    center_x = (left[0]+right[0])//2
    center_y = (top[1]+bottom[1])//2
    
    distance = pairwise.euclidean_distances([(center_x,center_y)],Y=[left,right,top,bottom])[0]
    max_distance = distance.max()
    radius = int(0.9*max_distance)
    circum = 2*np.pi*radius
    
    cir_roi = np.zeros(th_img.shape[:2],dtype='uint8')
    cv2.circle(cir_roi,(center_x,center_y),radius,255,10)
    cir_roi = cv2.bitwise_and(th_img,th_img,mask=cir_roi)
    
    image,contours,hierarchy = cv2.findContours(cir_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    count = 0
    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        out_of_bound = center_y*1.25 > y+h
        limit_points = circum*0.25 > cnt.shape[0]
        
        if out_of_bound and limit_points:
            count+=1
    return count

In [8]:
cam = cv2.VideoCapture(0)
num_frames = 0
while True:
    ret, frame = cam.read(0)
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames<60:
        calc_avg(gray,average_weight)
        if num_frames<=59:
            cv2.putText(frame_copy,'WAIT. GETTING BACKGROUND',(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count',frame_copy)
    else:
        hand = segment(gray)
        if hand is not None:
            th_img,hand_segment = hand
            
            #DRAWS CONTOUR AROUND REAL HAND
            cv2.drawContours(frame_copy,[hand_segment+(roi_right,roi_top)],-1,(255,0,0),3)
            fingers = count_fingers(th_img,hand_segment)
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Thresholded',th_img)
    
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    num_frames+=1
    cv2.imshow('Finger Count',frame_copy)
    k = cv2.waitKey(1) & 0xFF
    if k==27:
        break
cam.release()
cv2.destroyAllWindows()

'3.4.2'